In [10]:
# Please execute/shift-return this cell everytime you run the notebook.  Don't edit it. 
%load_ext autoreload
%autoreload 2
from notebook import * 

# Performance

## How programming languages affects performance?

### Python
    

In [18]:
render_code("./programming_languages/hello.py")

// ./programming_languages/hello.py:1-1 (1 lines)
print("Hello, World!")

In [19]:
!perf stat  taskset -c 8 python ./programming_languages/hello.py

Hello, World!

 Performance counter stats for 'taskset -c 8 python ./programming_languages/hello.py':

             20.70 msec task-clock                       #    0.942 CPUs utilized             
                 4      context-switches                 #  193.277 /sec                      
                 1      cpu-migrations                   #   48.319 /sec                      
               969      page-faults                      #   46.821 K/sec                     
     <not counted>      cpu_atom/cycles/                                                        (0.00%)
        31,791,467      cpu_core/cycles/                 #    1.536 GHz                       
     <not counted>      cpu_atom/instructions/                                                  (0.00%)
        50,718,862      cpu_core/instructions/                                                
     <not counted>      cpu_atom/branches/                                                      (0.00%)
        10,697,

### Perl

In [22]:
render_code("./programming_languages/hello.pl")

// ./programming_languages/hello.pl:1-5 (5 lines)
#!/usr/bin/perl
use strict;
use warnings;

print "Hello, World!\n";

In [23]:
! perf stat  taskset -c 8 ./programming_languages/hello.pl

Hello, World!

 Performance counter stats for 'taskset -c 8 ./programming_languages/hello.pl':

              8.98 msec task-clock                       #    0.876 CPUs utilized             
                 4      context-switches                 #  445.354 /sec                      
                 1      cpu-migrations                   #  111.339 /sec                      
               373      page-faults                      #   41.529 K/sec                     
     <not counted>      cpu_atom/cycles/                                                        (0.00%)
         9,766,755      cpu_core/cycles/                 #    1.087 GHz                       
     <not counted>      cpu_atom/instructions/                                                  (0.00%)
        15,848,594      cpu_core/instructions/                                                
     <not counted>      cpu_atom/branches/                                                      (0.00%)
         3,218,273    

### Java

In [20]:
render_code("./programming_languages/HelloWorld.java")

// ./programming_languages/HelloWorld.java:1-8 (8 lines)
public class HelloWorld {

    public static void main(String[] args) {
        // Prints "Hello, World" to the terminal window.
        System.out.println("Hello, World");
    }

}

In [21]:
! cd ./programming_languages/; javac HelloWorld.java
! cd ./programming_languages/; perf stat  taskset -c 8 java HelloWorld

Hello, World

 Performance counter stats for 'taskset -c 8 java HelloWorld':

             30.27 msec task-clock                       #    0.831 CPUs utilized             
                93      context-switches                 #    3.072 K/sec                     
                 1      cpu-migrations                   #   33.031 /sec                      
             5,618      page-faults                      #  185.567 K/sec                     
     <not counted>      cpu_atom/cycles/                                                        (0.00%)
        76,062,108      cpu_core/cycles/                 #    2.512 GHz                       
     <not counted>      cpu_atom/instructions/                                                  (0.00%)
       104,248,879      cpu_core/instructions/                                                
     <not counted>      cpu_atom/branches/                                                      (0.00%)
        19,104,560      cpu_core/branche

### Go

In [24]:
render_code("./programming_languages/hello-world.go")

// ./programming_languages/hello-world.go:1-5 (5 lines)
package main
import "fmt"
func main() {
    fmt.Println("hello world")
}

In [72]:
! perf stat  taskset -c 8 /usr/local/go/bin/go run ./programming_languages/hello-world.go

hello world

 Performance counter stats for 'taskset -c 8 /usr/local/go/bin/go run ./programming_languages/hello-world.go':

             94.87 msec task-clock                       #    0.494 CPUs utilized             
             4,262      context-switches                 #   44.923 K/sec                     
                 1      cpu-migrations                   #   10.540 /sec                      
            10,646      page-faults                      #  112.213 K/sec                     
     <not counted>      cpu_atom/cycles/                                                        (0.00%)
       297,224,309      cpu_core/cycles/                 #    3.133 GHz                       
     <not counted>      cpu_atom/instructions/                                                  (0.00%)
       596,789,308      cpu_core/instructions/                                                
     <not counted>      cpu_atom/branches/                                                      (

In [28]:
!/usr/local/go/bin/go build ./programming_languages/hello-world.go
! perf stat  taskset -c 8 ./hello-world

hello world

 Performance counter stats for 'taskset -c 8 ./hello-world':

              4.51 msec task-clock                       #    0.271 CPUs utilized             
                56      context-switches                 #   12.415 K/sec                     
                 1      cpu-migrations                   #  221.696 /sec                      
               237      page-faults                      #   52.542 K/sec                     
     <not counted>      cpu_atom/cycles/                                                        (0.00%)
         4,473,885      cpu_core/cycles/                 #    0.992 GHz                       
     <not counted>      cpu_atom/instructions/                                                  (0.00%)
         6,566,780      cpu_core/instructions/                                                
     <not counted>      cpu_atom/branches/                                                      (0.00%)
         1,168,232      cpu_core/branches/ 

### C++

In [26]:
render_code("./programming_languages/hello.cpp")

// ./programming_languages/hello.cpp:1-8 (8 lines)
#include <iostream>
using namespace std;

int main() 
{
    cout << "Hello, World!";
    return 0;
}

In [27]:
! g++ -O4 ./programming_languages/hello.cpp -o ./programming_languages/hello_cpp
! perf stat  taskset -c 8 ./programming_languages/hello_cpp

Hello, World!
 Performance counter stats for 'taskset -c 8 ./programming_languages/hello_cpp':

              3.38 msec task-clock                       #    0.686 CPUs utilized             
                 3      context-switches                 #  887.578 /sec                      
                 1      cpu-migrations                   #  295.859 /sec                      
               193      page-faults                      #   57.101 K/sec                     
     <not counted>      cpu_atom/cycles/                                                        (0.00%)
         3,636,113      cpu_core/cycles/                 #    1.076 GHz                       
     <not counted>      cpu_atom/instructions/                                                  (0.00%)
         5,012,773      cpu_core/instructions/                                                
     <not counted>      cpu_atom/branches/                                                      (0.00%)
           906,301    

### C

In [29]:
render_code("./programming_languages/hello.c")

// ./programming_languages/hello.c:1-7 (7 lines)
#include <stdio.h>

int main()
{
    printf("Hello, world!\n");
    return 0;
}

In [30]:
! gcc -O4 ./programming_languages/hello.c -o ./programming_languages/hello_c
! perf stat  taskset -c 8 ./programming_languages/hello_c

Hello, world!

 Performance counter stats for 'taskset -c 8 ./programming_languages/hello_c':

              2.02 msec task-clock                       #    0.562 CPUs utilized             
                 3      context-switches                 #    1.488 K/sec                     
                 1      cpu-migrations                   #  496.015 /sec                      
               122      page-faults                      #   60.514 K/sec                     
     <not counted>      cpu_atom/cycles/                                                        (0.00%)
         2,132,784      cpu_core/cycles/                 #    1.058 GHz                       
     <not counted>      cpu_atom/instructions/                                                  (0.00%)
         2,375,398      cpu_core/instructions/                                                
     <not counted>      cpu_atom/branches/                                                      (0.00%)
           436,991     

## How programmers affects performance?

In [33]:
compare([do_render_code("./programmer/madd_A.c", lang="c++", show="main"),
do_render_code("./programmer/madd_B.c", lang="c++", show="main")])

In [35]:
! make -C programmer clean all
! echo "config,IC,Cycles,CPI,CycleTime,ET,DL1_Miss_Rate,DL1_MISSES,DL1_ACCESSES" > ./programmer/stats.csv
! echo "running version A"; echo -n "A_O0," >> ./programmer/stats.csv; ./programmer/madd_A 2>> ./programmer/stats.csv
! echo "running version B"; echo -n "B_O0," >> ./programmer/stats.csv; ./programmer/madd_B 2>> ./programmer/stats.csv
display_df_mono(render_csv("programmer/stats.csv",columns=["config","IC", "CPI", "CycleTime", "ET"]))

make: Entering directory '/nfshome/htseng/courses/CS203/demo/performance/programmer'
rm -f madd_A madd_B madd_change_freq madd_A_unroll *_O3 *~
#hotspot hotspot_omp
cc -O0 -DHAVE_LINUX_PERF_EVENT_H -g perfstats.c madd_A.c -o madd_A
cc -O0 -DHAVE_LINUX_PERF_EVENT_H -g perfstats.c madd_B.c -o madd_B
cc -DHAVE_LINUX_PERF_EVENT_H -g -O3 perfstats.c madd_A.c -o madd_A_O3
cc -DHAVE_LINUX_PERF_EVENT_H -g -O3 perfstats.c madd_B.c -o madd_B_O3
cc -O3 -DHAVE_LINUX_PERF_EVENT_H -g -DA madd_change_freq.c perfstats.o -o madd_change_freq
madd_change_freq.c: In function ‘main’:
madd_change_freq.c:18:3: warning: implicit declaration of function ‘change_cpufrequnecy’ []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wimplicit-function-declaration-Wimplicit-function-declaration]8;;]
   18 |   change_cpufrequnecy(freq);
      |   ^~~~~~~~~~~~~~~~~~~
madd_change_freq.c:49:3: warning: implicit declaration of function ‘restore_cpufrequnecy’ []8;;https://gcc.gnu.org/onlinedocs/gcc/Warni

,index,config,IC,CPI,CycleTime,ET
0,0,A_O0,2551275489,0.202586,0.192765,0.099631
1,1,B_O0,2558124994,1.470503,0.192881,0.725568


In [ ]:
! lscpu

<div class="alert alert-block alert-info">
    
### Question:

What does the programmer affect?
    
</div>

In [68]:
compare([do_render_code("../introduction/arraySort.cpp",lang="c++", show="main"),do_render_code("../introduction/arraySort.cpp",lang="c++", show="calculate_sum")])

In [71]:
! cd ../introduction/; make clean; make; echo "not sorting"; perf stat  taskset -c 8 ./arraySort 262144 1000 0; echo "sorting"; perf stat   taskset -c 8 ./arraySort 262144 1000 1

rm -f madd arraySort bitonic quicksort mm quicksort_cpu
gcc madd.c -o madd 
gcc calculate_sum.c -c 
g++ -O0 arraySort.cpp calculate_sum.o -o arraySort
/usr/local/cuda/bin/nvcc -O3 bitonic.cu -o bitonic
/usr/local/cuda/bin/nvcc -O3 quicksort.cu -o quicksort
gcc -O3 mm.c perfstats.o -o mm 
/usr/local/cuda/bin/nvcc -O3 quicksort_cpu.cpp -o quicksort_cpu
not sorting
sum = 127161197

 Performance counter stats for 'taskset -c 8 ./arraySort 262144 1000 0':

            611.56 msec task-clock                       #    0.999 CPUs utilized             
                 6      context-switches                 #    9.811 /sec                      
                 1      cpu-migrations                   #    1.635 /sec                      
               453      page-faults                      #  740.733 /sec                      
     <not counted>      cpu_atom/cycles/                                                        (0.00%)
     3,137,208,199      cpu_core/cycles/                 #  

<div class="alert alert-block alert-info">
    
### Question:

What does the programmer affect?
    
</div>

In [ ]:
render_code("./programmer/perfstats.c", lang="c++", show="change_cpufrequnecy")

In [ ]:
!cpupower frequency-info -n

In [ ]:
! cat /proc/cpuinfo|grep MHz

In [ ]:
! cs203 job run "perf stat ./programmer/madd_change_freq 8192 800"
! cs203 job run "perf stat ./programmer/madd_change_freq 8192 1700"
! cs203 job run "perf stat ./programmer/madd_change_freq 8192 3300"

## Compiler

In [26]:
! cd programmer; make clean; make madd_A
! echo "running version A (Unoptimized)"; time ./programmer/madd_A

rm -f madd_A madd_B madd_change_freq madd_A_unroll *_O3 *~
#hotspot hotspot_omp
cc -O0 -DHAVE_LINUX_PERF_EVENT_H -g  madd_A.c -o madd_A
running version A (Unoptimized)
161

real	0m1.676s
user	0m1.524s
sys	0m0.151s


In [27]:
! cd programmer; make clean; make madd_B
! echo "running version B (Unoptimized)"; time ./programmer/madd_B

rm -f madd_A madd_B madd_change_freq madd_A_unroll *_O3 *~
#hotspot hotspot_omp
cc -O0 -DHAVE_LINUX_PERF_EVENT_H -g  madd_B.c -o madd_B
running version B (Unoptimized)
161

real	0m2.574s
user	0m2.436s
sys	0m0.136s


In [39]:
! cd programmer; rm madd_A_O3; make madd_A_O3
! echo "running version A (Optimized)"; echo -n "A_O3," >> ./programmer/stats.csv; time ./programmer/madd_A_O3  2>> ./programmer/stats.csv

cc -DHAVE_LINUX_PERF_EVENT_H -g -O3 perfstats.c madd_A.c -o madd_A_O3
running version A (Optimized)
161

real	0m1.543s
user	0m1.375s
sys	0m0.160s


In [42]:
! cd programmer; rm madd_B_O3; make madd_B_O3
! echo "running version B (Optimized)"; echo -n "B_O3," >> ./programmer/stats.csv; time ./programmer/madd_B_O3 2>> ./programmer/stats.csv

cc -DHAVE_LINUX_PERF_EVENT_H -g -O3 perfstats.c madd_B.c -o madd_B_O3
running version B (Optimized)
161

real	0m2.174s
user	0m1.980s
sys	0m0.183s


In [43]:
display_df_mono(render_csv("programmer/stats.csv",columns=["config","IC", "CPI", "CycleTime", "ET"]))

,index,config,IC,CPI,CycleTime,ET
0,0,A_O0,2551275489,0.202586,0.192765,0.099631
1,1,B_O0,2558124994,1.470503,0.192881,0.725568
2,2,A_O3,117874802,1.400096,0.192691,0.031801
3,3,A_O3,117888803,1.413868,0.192693,0.032118
4,4,B_O3,611200962,5.704716,0.193256,0.673831
5,5,B_O3,611378079,5.690006,0.193303,0.672452


<div class="alert alert-block alert-info">
    
### Question:

What's the limitation of compiler?
    
</div>

In [44]:
compare([do_render_code("compiler/arithmetics.c", show="a_mul_43"), do_render_code("compiler/arithmetics.c", show="a_mul_43_no_opt")])

In [45]:
render_code("compiler/arithmetics.c", show="main")
! cd compiler; gcc -O3 -DHAVE_LINUX_PERF_EVENT_H perfstats.c arithmetics.c -o arithmetics
! echo "IC,Cycles,CPI,CycleTime,ET,DL1_Miss_Rate,DL1_MISSES,DL1_ACCESSES" > ./compiler/stats.csv
! ./compiler/arithmetics 2>> compiler/stats.csv
display_df_mono(render_csv("compiler/stats.csv",columns=["IC", "CPI", "CycleTime", "ET"]))

// compiler/arithmetics.c:32-73 (42 lines)
int main(int argc, void **argv)
{
    clock_t start, stop;
    unsigned N = 16777216;
    int64_t *data, *data_2, i, r;
    if(argc > 1)
        N = atoi(argv[1]);
    data = (int64_t *)malloc(sizeof(int64_t) *N);
    data_2 = (int64_t *)malloc(sizeof(int64_t) *N);
    memcpy(data, data_2, sizeof(int64_t) *N);
    r = rand();
    start = clock();
    perfstats_init();
    perfstats_enable();
    for(i=0;i<N;i++)
        data[i] = a_mul_43_no_opt(data[i]);
    perfstats_disable();
    perfstats_print();
    perfstats_deinit();
    stop = clock();
    printf("No optimization\n");
    print_elapsed(start, stop);
    start = clock();
    perfstats_init();
    perfstats_enable();
    for(i=0;i<N;i++)
        data_2[i] = a_mul_43(data_2[i]);
    perfstats_disable();
    perfstats_print();
    perfstats_deinit();
    stop = clock();
    printf("Optimized\n");
    print_elapsed(start, stop);
    r = r%N;
    printf("%ld\n",a_mul_43(data[r]));
    printf("%ld\n",a_div_43(data[r]));
    printf("%ld\n",a_mod_43(data[r]));
    printf("%ld\n",a_mul_43(data_2[r]));
    printf("%ld\n",a_div_43(data_2[r]));
    printf("%ld\n",a_mod_43(data_2[r]));    
    return 0;
}

No optimization
Elapsed time: 0.015s
Optimized
Elapsed time: 0.033s
0
0
0
0
0
0


,index,IC,CPI,CycleTime,ET
0,0,335713224,0.223388,0.193228,0.014491
1,1,303660926,0.556847,0.193586,0.032734
